# Import
Import the dataset, rename some columns, convert units, and save in a optimized format

### Install Prerequisites
Then load libraries

In [1]:
# Run pip to install prerequisites
!pip install pandas termcolor
import pandas as pd
from termcolor import colored
import zipfile
import os

### Extract data
It comes packaged in a zip file, extract if necessary

In [2]:
# Look for this file in our cwd
dataname = 'data/household_power_consumption.txt'

# Extract zip if necessary
if not os.path.isfile(dataname):
    zipname = dataname.replace('data', '../src').replace('.txt', '.zip')
    try: 
        zip_ref = zipfile.ZipFile(zipname, 'r')
        zip_ref.extractall('data')
        zip_ref.close()
    except FileNotFoundError as e:
        print(colored("Source zipfile not found: \n\t{}".format(e), 'red'))
        raise e
        
# Use the unix head command to show the source data file
! head data/household_power_consumption.txt 

### Import Data
Load csv into a pandas dataframe

In [3]:
# Use pandas' read_csv, but don't worry about dates here, it seems to be much faster to do it after
df = pd.read_csv(
    dataname,
    delimiter=';',
    na_values='?',
    header=0,
    names=['date', 'time', 'active', 'reactive', 'volts', 'amps', 'sub1', 'sub2', 'sub3'],
    memory_map=True,
)

In [4]:
# Now fix the dates and times, combine into one column
df['dtime'] = pd.to_datetime(df.date + ' ' + df.time, dayfirst=True, infer_datetime_format=True)

# And index on it
df.set_index('dtime', inplace=True, drop=True)

In [5]:
# Don't need the date and time columns now
df.drop(columns=['date', 'time'], inplace=True)

### Conversions
Match units across features

In [6]:
# Subs 1,2,3 are in Wh/min -- match with active power (kW) by multiplying by 60/1000
to_kW = lambda index: index * 60 / 1000

df.sub1 = to_kW(df.sub1)
df.sub2 = to_kW(df.sub2)
df.sub3 = to_kW(df.sub2)

In [7]:
# Convert timezone, assume 'UTC'
df.tz_localize('UTC', copy=False)

,active,reactive,volts,amps,sub1,sub2,sub3
dtime,,,,,,,
2006-12-16 17:24:00+00:00,4.216,0.418,234.84,18.4,0.0,0.06,0.0036
2006-12-16 17:25:00+00:00,5.360,0.436,233.63,23.0,0.0,0.06,0.0036
2006-12-16 17:26:00+00:00,5.374,0.498,233.29,23.0,0.0,0.12,0.0072
2006-12-16 17:27:00+00:00,5.388,0.502,233.74,23.0,0.0,0.06,0.0036
2006-12-16 17:28:00+00:00,3.666,0.528,235.68,15.8,0.0,0.06,0.0036
2006-12-16 17:29:00+00:00,3.520,0.522,235.02,15.0,0.0,0.12,0.0072
2006-12-16 17:30:00+00:00,3.702,0.520,235.09,15.8,0.0,0.06,0.0036
2006-12-16 17:31:00+00:00,3.700,0.520,235.22,15.8,0.0,0.06,0.0036
2006-12-16 17:32:00+00:00,3.668,0.510,233.99,15.8,0.0,0.06,0.0036


### Export

In [8]:
df.to_pickle('data/imported.pickle')